In [4]:
import pandas as pd
import numpy as np
import csv
import math

In [5]:
def change_feature(x):
    x = x.drop(['LIMIT_BAL'],axis = 1)
    x['female'] = np.where(x['SEX']==2, 1, 0)
    x['male'] = np.where(x['SEX']==1, 1, 0)
    for i in range(6):
        x[i+1] = np.where(x['EDUCATION']==i+1, 1, 0)
    x['Marry'] = np.where(x['MARRIAGE']==2, 1, 0)
    x['Single'] = np.where(x['MARRIAGE']==1, 1, 0)
    x['Unknown'] = np.where(x['MARRIAGE']==3, 1, 0)
    x = x.drop(['EDUCATION','SEX','MARRIAGE'],axis = 1)
    #    x= x.drop(['SEX','EDUCATION','MARRIAGE'],axis = 1)
#    x['education^2']=x['EDUCATION']**2
#    x['Bill_sum']=x['BILL_AMT1']+x['BILL_AMT2']+x['BILL_AMT3']+x['BILL_AMT4']+x['BILL_AMT5']+x['BILL_AMT6']
#    x['PAY_AMT_sum'] = 0
 #   for i in x:
  #      if 'PAY_AMT' in i:
   #         x['PAY_AMT_sum']=x[i]+x['PAY_AMT_sum']
    return x

In [6]:
def training(inp_csvx,inp_csvy):
    x_train = pd.read_csv(inp_csvx)
    y_train = pd.read_csv(inp_csvy)
    x_train = change_feature(x_train)
    print (x_train.columns)
    X = x_train.values
    Y = y_train.T.values[0].astype(float)

    #feature scaling
    minx = np.min(X, axis=0)
    maxx = np.max(X, axis=0)
    X = (X - minx) / (maxx - minx)
    #optimize
    
    w =  np.zeros(len(X[0]))
    m = X.shape[0]
    b = 0
    lamba = 10^-6
    
    lr = (2.81)*(10^-5)
    lr_b = 0
    lr_w = 0
    current_F1 = 0
    run = 10200
    costs = []
    count = 0
    regul_p = 10^-9
    for i in range(run):
        
        tem_y = np.dot(X,w)+b
        h0 = 1.0/(1 + np.exp(-1.0 * tem_y))
#        cost = (-1.0) * np.mean(np.multiply(Y, np.log(h0)) + np.multiply(1.0-Y, np.log(1.0 - h0)))
        dw = (np.dot(X.T, np.transpose(h0 - Y)) + regul_p * w) / m  
        db = np.mean((h0 - Y))

        lr_w += dw ** 2
        lr_b += db ** 2
#
        w = w - lr * dw / (np.sqrt(lr_w)+lamba)
        b = b - lr * db / (np.sqrt(lr_b)+lamba)

#        costs.append(cost)
        count +=1
        if count%1000 == 0:
            print(('run %d times' %count))

    return w,b,minx,maxx

In [7]:
def test(test_csv,output_csv,a):
    w=a[0]
    b=a[1]
    minx = a[2]
    maxx = a[3]
    test = pd.read_csv(test_csv)
    test = change_feature(test)
    Xt = test.values
    Xt = (Xt - minx) / (maxx - minx)
    Yt = np.dot(Xt,w)+b
    h0t = 1.0/(1 + np.exp(-1.0 * Yt))

    Prediction = [1 if i >= 0.5 else 0 for i in h0t]
    id_ = np.array([])
    for i in range(len(Yt)):
        id_=np.append(id_,'id_'+str(i))
    final = pd.DataFrame(np.concatenate(([id_],[Prediction])).transpose(),columns = ['id','Value'])
    final.to_csv(output_csv,index = False)
    return Prediction

In [8]:
#train_x=input()
#train_y=input()

train_x = 'train_x.csv'
train_y = 'train_y.csv'

#train_x,train_y,test_x,out = input().split()

a = training(train_x,train_y)
test_x = 'test_x.csv'
out = 'sample_submission_.csv'
t=test(test_x,out,a)

Index([      'AGE',     'PAY_0',     'PAY_2',     'PAY_3',     'PAY_4',
           'PAY_5',     'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
       'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',  'PAY_AMT1',  'PAY_AMT2',
        'PAY_AMT3',  'PAY_AMT4',  'PAY_AMT5',  'PAY_AMT6',    'female',
            'male',           1,           2,           3,           4,
                 5,           6,     'Marry',    'Single',   'Unknown'],
      dtype='object')
run 1000 times
run 2000 times
run 3000 times
run 4000 times
run 5000 times
run 6000 times
run 7000 times
run 8000 times
run 9000 times
run 10000 times


In [9]:
sum(t)

972

In [96]:
a

(array([  6.21325672e-01,   6.65830059e+00,   3.22564932e-01,
          2.42282153e+00,  -6.06901355e-01,  -2.04043220e-02,
         -2.29332771e+00,  -1.12747533e+02,   4.13105341e+01,
         -2.35349643e+01,   6.55681130e+00,   1.38933389e+02,
         -2.33787732e+01,  -2.85910563e+02,  -2.03251608e+02,
         -1.20779716e+02,  -1.50688439e+02,  -1.88484059e+02,
         -1.34909596e+02,  -2.25082791e+01,  -2.25628911e+01,
         -2.27052448e+01,  -1.16041378e+02,  -1.29600852e+02,
         -4.00916999e+01,   1.78479183e+01,   1.79931485e+01,
          1.75400521e+01]),
 15.547747035061233,
 array([     21,      -2,      -2,      -2,      -2,      -2,      -2,
        -165580,  -69777, -157264, -170000,  -61372, -339603,       0,
              0,       0,       0,       0,       0,       0,       0,
              0,       0,       0,       0,       0,       0,       0], dtype=int64),
 array([     79,       8,       7,       8,       8,       8,       8,
         964511,  98393

In [43]:
a

(array([  0.35664212,   6.14401107,   0.63895967,   1.09383371,
         -0.26167628,   0.16108683,  -0.35113353, -20.74591359,
          7.41897882,  -4.56342781,   0.18006909,  22.60755843,
         -2.92350853, -48.3786634 , -36.67192043, -21.20054761,
        -25.01792622, -26.82800533, -18.77956817,  -5.5844293 ,
         -5.66560474,  -5.74220427, -13.75391427, -10.99803279,
         -6.96250795,   3.40921941,   3.59086607,   3.5915781 ]),
 2.0615917290381667,
 array([     21,      -2,      -2,      -2,      -2,      -2,      -2,
        -165580,  -69777, -157264, -170000,  -61372, -339603,       0,
              0,       0,       0,       0,       0,       0,       0,
              0,       0,       0,       0,       0,       0,       0], dtype=int64),
 array([     79,       8,       7,       8,       8,       8,       8,
         964511,  983931, 1664089,  891586,  927171,  961664,  873552,
        1684259,  896040,  621000,  417990,  528666,       1,       1,
              1, 

In [49]:
a

(array([  0.35664212,   6.14401107,   0.63895967,   1.09383371,
         -0.26167628,   0.16108683,  -0.35113353, -20.74591359,
          7.41897882,  -4.56342781,   0.18006909,  22.60755843,
         -2.92350853, -48.3786634 , -36.67192043, -21.20054761,
        -25.01792622, -26.82800533, -18.77956817,  -5.5844293 ,
         -5.66560474,  -5.74220427, -13.75391427, -10.99803279,
         -6.96250795,   3.40921941,   3.59086607,   3.5915781 ]),
 2.0615917290381667,
 array([     21,      -2,      -2,      -2,      -2,      -2,      -2,
        -165580,  -69777, -157264, -170000,  -61372, -339603,       0,
              0,       0,       0,       0,       0,       0,       0,
              0,       0,       0,       0,       0,       0,       0], dtype=int64),
 array([     79,       8,       7,       8,       8,       8,       8,
         964511,  983931, 1664089,  891586,  927171,  961664,  873552,
        1684259,  896040,  621000,  417990,  528666,       1,       1,
              1, 